In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense

In [ ]:
data_inmet = pd.read_csv('../../model/additional_data/datasets/inmet_2019_2024.csv')
data_modis = pd.read_csv('../../model/data/modisTodos.csv')

In [5]:
# Verificar os tipos de dados das colunas de latitude e longitude
print("Tipos de dados em data_inmet:")
print(data_inmet.dtypes)
print("\nTipos de dados em data_modis:")
print(data_modis.dtypes)

Tipos de dados em data_inmet:
Data                                                      object
latitude                                                  object
longitude                                                 object
PRECIPITACAO TOTAL, HORARIO (mm)                         float64
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    float64
RADIACAO GLOBAL (KJ/m2)                                  float64
TEMPERATURA DO AR - BULBO SECO, HORARIA (0C)             float64
UMIDADE RELATIVA DO AR, HORARIA (%)                      float64
VENTO, RAJADA MAXIMA (m/s)                               float64
dtype: object

Tipos de dados em data_modis:
latitude      float64
longitude     float64
confidence    float64
dtype: object


In [6]:
# Verificar se há valores nulos
print("\nValores nulos em data_inmet:")
print(data_inmet[['latitude', 'longitude']].isnull().sum())
print("\nValores nulos em data_modis:")
print(data_modis[['latitude', 'longitude']].isnull().sum())


Valores nulos em data_inmet:
latitude     0
longitude    0
dtype: int64

Valores nulos em data_modis:
latitude     0
longitude    0
dtype: int64


In [7]:
# Converter as colunas para string, se necessário
data_inmet['latitude'] = data_inmet['latitude'].astype(str).str.replace(',', '.')
data_inmet['longitude'] = data_inmet['longitude'].astype(str).str.replace(',', '.')

data_modis['latitude'] = data_modis['latitude'].astype(str).str.replace(',', '.')
data_modis['longitude'] = data_modis['longitude'].astype(str).str.replace(',', '.')

In [8]:
# Ajustar os tipos de dados para latitude e longitude
data_inmet['latitude'] = data_inmet['latitude'].astype(float)
data_inmet['longitude'] = data_inmet['longitude'].astype(float)
data_modis['latitude'] = data_modis['latitude'].astype(float)
data_modis['longitude'] = data_modis['longitude'].astype(float)

In [9]:
# Arredondar coordenadas para combinar dados com tolerância
data_inmet['latitude'] = data_inmet['latitude'].round(4)
data_inmet['longitude'] = data_inmet['longitude'].round(4)
data_modis['latitude'] = data_modis['latitude'].round(4)
data_modis['longitude'] = data_modis['longitude'].round(4)

In [10]:
# Preparar os dados para Nearest Neighbors
inmet_coords = data_inmet[['latitude', 'longitude']].values
modis_coords = data_modis[['latitude', 'longitude']].values
nbrs = NearestNeighbors(n_neighbors=1, radius=0.01).fit(modis_coords)
distances, indices = nbrs.kneighbors(inmet_coords)

In [11]:
# Adicionar índice de correspondência e distância ao DataFrame do INMET
data_inmet['modis_index'] = indices.flatten()
data_inmet['distance'] = distances.flatten()

In [12]:
# Filtrar correspondências dentro da tolerância
tolerance = 0.1
filtered_inmet = data_inmet[data_inmet['distance'] < tolerance]

In [13]:
# Fazer o merge
combined_data_tolerance = pd.merge(filtered_inmet, data_modis, left_on='modis_index', right_index=True, how='inner')

# Verifique o DataFrame combinado com tolerância
print("DataFrame Combinado com Tolerância:")
print(combined_data_tolerance.head())

DataFrame Combinado com Tolerância:
         Data  latitude_x  longitude_x  PRECIPITACAO TOTAL, HORARIO (mm)  \
0  2019-01-01    -15.7893     -47.9258                               1.4   
1  2019-01-02    -15.7893     -47.9258                               0.0   
2  2019-01-03    -15.7893     -47.9258                               0.0   
3  2019-01-04    -15.7893     -47.9258                               0.0   
4  2019-01-05    -15.7893     -47.9258                               1.0   

   PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)  \
0                                             888.02       
1                                             888.80       
2                                             887.62       
3                                             887.54       
4                                             887.96       

   RADIACAO GLOBAL (KJ/m2)  TEMPERATURA DO AR - BULBO SECO, HORARIA (0C)  \
0                  1192.79                                         20.

In [14]:
# Definir X e y para treinamento (usando variáveis importantes como latitude, longitude e outros dados climáticos)
X = combined_data_tolerance[['latitude_x', 'longitude_x']]
y = combined_data_tolerance['confidence']  # Defina a coluna correta de saída

In [15]:
# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Criar o modelo Feedforward
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1)

NameError: name 'Sequential' is not defined

In [ ]:
# Fazer previsões
predictions = model.predict(X_test).flatten()

Epoch 1/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 315us/step - loss: 1718.4664
Epoch 2/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 304us/step - loss: 868.7367
Epoch 3/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 323us/step - loss: 834.3995
Epoch 4/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 300us/step - loss: 819.9442
Epoch 5/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 300us/step - loss: 811.8189
Epoch 6/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 298us/step - loss: 814.2697
Epoch 7/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 299us/step - loss: 808.2418
Epoch 8/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 301us/step - loss: 798.2357
Epoch 9/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 300us/step - loss: 809.6699
Epoch 10/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 300us/step - loss: 791.7117
Epoch 11/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 300us/step - loss: 789.5043
Epoch 12/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 301us/step - loss: 785.4346
Epoch 13/100
3739/3739 ━━━━━━━━━━━━━━━━━━━━ 1s 322us/step - loss: 782.5067
Epoch 14/100
3739/3739 ━━━━━━━━━━

In [ ]:
# Gerar datas futuras a partir de hoje
hoje = pd.to_datetime("2024-11-04")
datas_futuras = pd.date_range(hoje, periods=len(predictions), freq='D')
datas_repetidas = np.tile(datas_futuras, int(np.ceil(len(predictions) / len(datas_futuras))))[:len(predictions)]

293/293 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step


In [ ]:
# Verifique se o DataFrame combinado não está vazio
if not combined_data_tolerance.empty:
    # Copiar as latitudes e longitudes combinadas para o DataFrame final
    final_df = combined_data_tolerance[['latitude_x', 'longitude_x']].iloc[:len(predictions)].copy()

    # Ajuste de `predictions` e `datas_repetidas` para o tamanho correto
    if len(predictions) < len(final_df):
        repeated_predictions = np.tile(predictions, int(np.ceil(len(final_df) / len(predictions))))[:len(final_df)]
    else:
        repeated_predictions = predictions[:len(final_df)]

    if len(datas_repetidas) < len(final_df):
        repeated_dates = np.tile(datas_repetidas, int(np.ceil(len(final_df) / len(datas_repetidas))))[:len(final_df)]
    else:
        repeated_dates = datas_repetidas[:len(final_df)]

    # Adicionar colunas de previsões e datas ao DataFrame final
    final_df['predicted_confidence'] = repeated_predictions
    final_df['Data'] = repeated_dates

    # Resetar o índice do DataFrame final
    final_df.reset_index(drop=True, inplace=True)

    # Reorganizar colunas para que 'Data' fique em primeiro lugar
    final_df = final_df[['Data', 'latitude_x', 'longitude_x', 'predicted_confidence']]

    # Exibir o DataFrame final
    print("DataFrame final com latitude, longitude, predicted_confidence e data:")
    print(final_df.head())
else:
    print("O DataFrame combinado está vazio. Não há dados para criar o DataFrame final.")

In [ ]:
# Salvar o DataFrame em um arquivo CSV
final_df.to_csv('previsao_confidence_feedforward.csv', index=False)

             data  latitude_x  longitude_x  PRECIPITACAO TOTAL, HORARIO (mm)  \
0      2019-01-01    -15.7893     -47.9258                               1.4   
1      2019-01-02    -15.7893     -47.9258                               0.0   
2      2019-01-03    -15.7893     -47.9258                               0.0   
3      2019-01-04    -15.7893     -47.9258                               0.0   
4      2019-01-05    -15.7893     -47.9258                               1.0   
...           ...         ...          ...                               ...   
48803  2024-08-27    -27.6025     -48.6200                               0.0   
48804  2024-08-28    -27.6025     -48.6200                               0.0   
48805  2024-08-29    -27.6025     -48.6200                               0.0   
48806  2024-08-30    -27.6025     -48.6200                               0.0   
48807  2024-08-31    -27.6025     -48.6200                               0.0   

       PRESSAO ATMOSFERICA AO NIVEL DA 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

DataFrame final com latitude, longitude e predicted_confidence:
   latitude_x  longitude_x  predicted_confidence
0    -15.7893     -47.9258             29.565857
1    -15.7893     -47.9258             45.698273
2    -15.7893     -47.9258             34.051170
3    -15.7893     -47.9258             27.683376
4    -15.7893     -47.9258             38.915058


In [ ]:
# Carregar os dados
data_modis = pd.read_csv('../../model/data/modisTodos.csv')
data_inmet = pd.read_csv('../../model/additional_data/datasets/inmet_2019_2024.csv')
resultados_novos = pd.read_csv('./previsao_confidence_feedforward.csv')

In [ ]:
# Configuração de estilo
sns.set(style="whitegrid")

In [ ]:
# Garantindo que as datas estão no formato correto e ordenando os dados
resultados_novos['Data'] = pd.to_datetime(resultados_novos['Data'])
resultados_novos = resultados_novos.sort_values(by='Data')

# Configuração do gráfico
plt.figure(figsize=(10, 6))
# Use 'predicted_confidence' se esse for o nome correto da coluna
sns.lineplot(data=resultados_novos, x='Data', y='predicted_confidence', color='red', ci=None)  # ci=None remove as margens

# Ajuste dos títulos e eixos
plt.title('Previsões de Probabilidade de Queimada - Modelo FeedForward')
plt.xlabel('Data')
plt.ylabel('Probabilidade de Queimada')

# Exibir o gráfico
plt.xticks(rotation=45)  # Rotaciona as datas para melhor legibilidade
plt.tight_layout()  # Ajusta o layout
plt.show()